# 🏛️ Italian Parliament Speech Analyzer

This notebook runs the complete analysis pipeline for Italian Parliament speeches.

## What this notebook does:
1. **Scrapes** speeches from senato.it and camera.it
2. **Generates embeddings** using Sentence Transformers
3. **Computes analytics** (identity, relations, temporal, sentiment)
4. **Exports JSON** files for the frontend visualization

---

⚠️ **GPU Recommended**: Enable GPU runtime for faster embedding generation.

`Runtime → Change runtime type → T4 GPU`

## 1. Setup Environment

In [2]:
# Clone the repository
!git clone https://github.com/WeridFire/Parliament-Speech-Analyzer.git

fatal: destination path 'Parliament-Speech-Analyzer' already exists and is not an empty directory.


In [6]:
# Install Python dependencies
%cd /content/Parliament-Speech-Analyzer
!pip install -r requirements.txt
# Install spaCy for advanced keyword extraction
!pip install -q spacy
!python -m spacy download it_core_news_sm -q

print("✅ Core dependencies installed!")

[Errno 2] No such file or directory: 'Parliament-Speech-Analyzer'
/content/Parliament-Speech-Analyzer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 18.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
✅ Core dependencies installed!


In [1]:
# Optional: Install transformer sentiment (more accurate, takes longer)
# Comment the line below if you don't want to use --transformer-sentiment

!pip install -q transformers torch

USE_TRANSFORMER_SENTIMENT = True  # Set to True if you installed transformers

## 2. Configuration

Choose your data source and analysis settings.

In [ ]:
# Configuration
DATA_SOURCE = "both"  # Options: "senate", "camera", "both"
FORCE_REFETCH = False  # Set to True to re-scrape from parliament websites
FORCE_REEMBED = False  # Set to True to regenerate embeddings

print(f"📋 Configuration:")
print(f"   Data source: {DATA_SOURCE}")
print(f"   Force refetch: {FORCE_REFETCH}")
print(f"   Force reembed: {FORCE_REEMBED}")
print(f"   Transformer sentiment: {USE_TRANSFORMER_SENTIMENT}")

## 3. Run the Analysis Pipeline

This step:
- Scrapes speeches (first run takes ~10-15 minutes)
- Generates embeddings (~5 min with GPU, ~15 min CPU)
- Computes all analytics
- Exports JSON files

In [ ]:
# Build the command
cmd = f"python backend/export_data.py --source {DATA_SOURCE}"

if FORCE_REFETCH:
    cmd += " --refetch"
if FORCE_REEMBED:
    cmd += " --reembed"
if USE_TRANSFORMER_SENTIMENT:
    cmd += " --transformer-sentiment"

print(f"🚀 Running: {cmd}\n")
print("="*60)

# Run the pipeline
!{cmd}

## 4. Verify Output

Check that the JSON files were generated successfully.

In [ ]:
import os
import json

output_dir = "frontend/public"
files = [f for f in os.listdir(output_dir) if f.endswith('.json')]

print("📁 Generated files:")
for f in files:
    path = os.path.join(output_dir, f)
    size_mb = os.path.getsize(path) / (1024 * 1024)

    # Load and show stats
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    n_speeches = len(data.get('speeches', []))
    n_deputies = len(data.get('deputies', []))

    print(f"   ✅ {f}: {size_mb:.2f} MB | {n_speeches} speeches | {n_deputies} deputies")

## 5. Download Results

Download the generated JSON files to use with the frontend.

In [ ]:
from google.colab import files

# Create a zip of all output files
!cd frontend/public && zip -r ../../parliament_data.zip *.json

# Download the zip
files.download('parliament_data.zip')

print("\n📥 Download started! Extract the zip and place files in frontend/public/")

## 6. Quick Data Exploration

Preview some analytics results.

In [ ]:
import json

# Load one of the output files
with open('frontend/public/camera.json', 'r', encoding='utf-8') as f:
    camera_data = json.load(f)

# Show available analytics
analytics = camera_data.get('analytics', {}).get('global', {})
print("📊 Available Analytics:")
for key in analytics.keys():
    print(f"   - {key}")

In [ ]:
# Example: Show distinctive keywords for a party
identity = analytics.get('identity', {})
keywords = identity.get('distinctive_keywords', {})

print("\n🏷️ Distinctive Keywords by Party:")
for party, words in list(keywords.items())[:5]:  # First 5 parties
    print(f"\n   {party}:")
    print(f"   {', '.join(words[:10])}")

In [ ]:
# Example: Show party affinity matrix
relations = analytics.get('relations', {})
affinity = relations.get('affinity_matrix', {})

if affinity:
    print("\n🤝 Top Party Pairs by Semantic Similarity:")
    pairs = affinity.get('pairs', [])[:10]
    for p in pairs:
        print(f"   {p['party1']} ↔ {p['party2']}: {p['similarity']:.3f}")

---

## Next Steps

1. **Download** `parliament_data.zip` from Step 5
2. **Extract** the JSON files to your local `frontend/public/` folder
3. **Run the frontend** locally:
   ```bash
   cd frontend
   npm install
   npm run dev
   ```
4. **Open** `http://localhost:5173` in your browser

---

Made with ❤️ for political science research